In [1]:
import pandas as pd
from tqdm import tqdm
import json

In [2]:
temp = pd.read_csv("result.csv")
temp = temp[temp["dir_ratio"] <= 0.5]
EXCLUDE=temp["dataset"].unique().tolist()

In [3]:
# AdaBoost_raw = pd.read_csv("result_mp_adjust_adaboost.csv")
case0_raw = pd.read_csv("result_mp_adjust_case0.csv")
custom_raw = pd.read_csv("result_mp_adjust_product.csv")

# merge
df = pd.concat([case0_raw, custom_raw], ignore_index=True)
# remove invalid data
df = df[(df["dist_error"]!=0) | (df["dir_error"]!=0) | (df["dir_ratio"]!=0)]
df = df[~df["dataset"].isin(EXCLUDE)]

# select parameters that have `dist_ratio` > 0.9 on test_case0
acceptable = df[df["dataset"] == "../test_case0"]
acceptable = acceptable[acceptable["dir_ratio"] > 0.9]
acceptable = acceptable[[
    'model_name',
    'distance_frac_step',
    'clean_data',
    'optimized_mode_ratio',
    'butter_Wn']]
# acceptable = acceptable.drop_duplicates()
# acceptable = acceptable.values.tolist()

In [4]:
# compare by dir_ratio, test_case0 has a weight of 50, others have a weight of 1

# add weight
df["weight"] = 1
# df.loc[df["dataset"] == "../test_case0", "weight"] = 50

# compare
selected_args_ratio = []
max_mean_dir_ratio = 0
no_weight_ratio=0
for args in tqdm(acceptable[['clean_data','optimized_mode_ratio','butter_Wn']].drop_duplicates().values.tolist()):
    temp = df[
        (df["clean_data"] == args[0]) &
        (df["optimized_mode_ratio"] == args[1]) &
        (df["butter_Wn"] == args[2])
    ]
    if len(temp) == 0:
        continue
    mean_dir_ratio = (temp["dir_ratio"] * temp["weight"]).sum() / temp["weight"].sum()
    if mean_dir_ratio > max_mean_dir_ratio:
        max_mean_dir_ratio = mean_dir_ratio
        selected_args_ratio = args
        no_weight_ratio = temp["dir_ratio"].mean()

100%|██████████| 94/94 [00:00<00:00, 218.17it/s]


In [5]:
print('Best args: '+json.dumps({
    "clean_data": selected_args_ratio[0],
    "optimized_mode_ratio": selected_args_ratio[1],
    "butter_Wn": selected_args_ratio[2]
}, indent=4))
print('Mean dir_ratio (with weight): '+ str(max_mean_dir_ratio))
print('Mean dir_ratio (without weight): '+ str(no_weight_ratio))
df[
    (df["clean_data"] == selected_args_ratio[0]) &
    (df["optimized_mode_ratio"] == selected_args_ratio[1]) &
    (df["butter_Wn"] == selected_args_ratio[2])
][["dataset", "dist_error", "dir_error", "dir_ratio", "weight"]].sort_values('dir_ratio', ascending=False).reindex()

Best args: {
    "clean_data": 4.0,
    "optimized_mode_ratio": 0.8,
    "butter_Wn": 0.005
}
Mean dir_ratio (with weight): 0.6984412625867368
Mean dir_ratio (without weight): 0.6984412625867368


,dataset,dist_error,dir_error,dir_ratio,weight
792641,Pocket-Walk/Pocket-Walk-09-001,340.527542,5.937972,0.932990,1
783569,Pocket-Walk/Pocket-Walk-09-001,284.906641,5.937972,0.932990,1
793289,Pocket-Walk/Pocket-Walk-09-001,284.906641,5.937972,0.932990,1
791993,Pocket-Walk/Pocket-Walk-09-001,65.077592,5.937972,0.932990,1
791345,Pocket-Walk/Pocket-Walk-09-001,83.678269,5.937972,0.932990,1
...,...,...,...,...,...
300809,Hand-Walk/Hand-Walk-02-007,46.641574,21.997415,0.484472,1
304697,Hand-Walk/Hand-Walk-02-007,59.463297,21.997415,0.484472,1
305345,Hand-Walk/Hand-Walk-02-007,60.388011,21.997415,0.484472,1
305993,Hand-Walk/Hand-Walk-02-007,52.832007,21.997415,0.484472,1


In [9]:
len(pd.read_csv("result.csv")["dataset"])-len(EXCLUDE)

29